# 1. Import Libraries

In [2]:
# Auto reloads src modules.
%load_ext autoreload
%autoreload 2

proj_root = r'c:\\Users\\KMA62139\\OneDrive - Kia\\Documents - Big Data, Data Science\\Projects\\crnn-pytorch'
import sys
import os
os.chdir(proj_root)
if proj_root not in sys.path:
    sys.path.append(proj_root)

from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
from src.data import preprocess

tqdm.pandas()


# 2. Extract Metadata for the Raw Images

In [3]:

# Extracts metadata related to the file path, label, and image width + height.
wbsin_dir = Path().cwd() / "data" / "raw" / "WBSIN_corrected"
wbsin_meta_df = preprocess.extract_jpg_meta(img_dir=wbsin_dir, img_type="wbsin")
wbsin_meta_df["xml_path"] = wbsin_meta_df["file_path"].apply(
    lambda x: str(x).replace("jpg", "xml")
)

wbsin_meta_df["bnd_box"] = wbsin_meta_df["xml_path"].progress_apply(lambda x: preprocess.extract_bnd_box(x))

100%|██████████| 5000/5000 [00:05<00:00, 974.13it/s] 


In [5]:
wbsin_meta_df["file_path"].iloc[0]

WindowsPath('c:/Users/KMA62139/OneDrive - Kia/Documents - Big Data, Data Science/Projects/crnn-pytorch/data/raw/WBSIN_corrected/KVD1_WBSIN_20210305_1000_3KPA24AD6LE328168.jpg')

# 3. Crop Images with Bounding Boxes

In [24]:
wbsin_meta_df["crop_path"] = wbsin_meta_df.progress_apply(
    lambda row: preprocess.crop_and_save(
        row["file_path"],
        row["label"],
        row["bnd_box"]["xmin"],
        row["bnd_box"]["xmax"],
        row["bnd_box"]["ymin"],
        row["bnd_box"]["ymax"],
        Path.cwd() / "data" / "interim" / "cropped_wbsin_corrected_images",
    ),
    axis=1,
)


wbsin_meta_df["crop_height"] = wbsin_meta_df.progress_apply(
    lambda row: row["bnd_box"]["ymax"] - row["bnd_box"]["ymin"], axis=1
)
wbsin_meta_df["crop_width"] = wbsin_meta_df.progress_apply(
    lambda row: row["bnd_box"]["xmax"] - row["bnd_box"]["xmin"], axis=1
)

wbsin_meta_df["crop_ratio"] = wbsin_meta_df["crop_width"] / wbsin_meta_df["crop_height"]


100%|██████████| 5000/5000 [00:00<00:00, 58134.72it/s]


In [26]:
wbsin_meta_df["proc_path"] = wbsin_meta_df.progress_apply(
    lambda row: preprocess.preprocess_cropped_image(
        str(row["crop_path"]),
        row["label"],
        Path.cwd() / "data" / "processed" / "wbsin_corrected_images",
    ),
    axis=1,
)

100%|██████████| 5000/5000 [51:42<00:00,  1.61it/s]


In [ ]:
# Saves the extracted metadata.
interim_path = Path.cwd() / "data" / "processed" 
interim_path.mkdir(parents=True, exist_ok=True)
wbsin_meta_df.to_csv(interim_path / "wbsin_corrected_processed_meta.csv", index=False, mode="w+")
wbsin_meta_df

In [ ]:
cropped_wbsin_meta_df = pd.read_csv(interim_path / "cropped_wbsin_meta.csv")
sample_crop_path_arr = cropped_wbsin_meta_df["crop_path"].head(5)

for sample_crop_path in sample_crop_path_arr:

    # Plots original image
    img = cv2.imread(sample_crop_path,0)

    plt.imshow(img)
    plt.show()

    # Plots the adaptive threshold image.
    img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, blockSize=3,C=3)
    plt.imshow(img)
    plt.show()


    # Plots the contour dilated image.
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 1))
    img = cv2.dilate(img, kernel)
    plt.imshow(img)
    plt.show()

    # Plots the denoised Gaussian blurred image
    blur = cv2.GaussianBlur(img,(3,3),0)
    img = cv2.threshold(blur, 100, 255, cv2.THRESH_BINARY)[1]
    plt.imshow(img)
    plt.show()

    # Plots the thinned image.
    kernel = np.ones((3,3),np.uint8)
    img = cv2.erode(img,kernel,iterations = 2)
    plt.imshow(img)
    plt.show()

    # Fixes skew.
    delta = .05
    limit = 4
    angles = np.arange(0, limit+delta, delta)
    scores = []
    for angle in angles:
        hist, score = find_score(img, angle)
        scores.append(score)
    best_score = max(scores)
    best_angle = angles[scores.index(best_score)]
    best_angle = round(best_angle, 2)
    print('Best angle: {}'.format(best_angle))

 #   img = img.rotate(best_angle, PIL.image.NEAREST, expand=1, fillcolor="white")

    img = im.fromarray(img)
    img = img.rotate(best_angle, expand=1, fillcolor="white")
 

    plt.imshow(img)
    plt.show()


